# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 10:08AM,238254,10,0085717625,ISDIN Corporation,Released
1,May 27 2022 10:08AM,238254,10,0085717686,ISDIN Corporation,Released
2,May 27 2022 10:08AM,238254,10,0085717713,ISDIN Corporation,Released
3,May 27 2022 10:08AM,238254,10,0085717714,ISDIN Corporation,Released
4,May 27 2022 10:08AM,238254,10,0085717715,ISDIN Corporation,Released
5,May 27 2022 10:08AM,238254,10,0085717723,ISDIN Corporation,Released
6,May 27 2022 10:08AM,238254,10,0085717725,ISDIN Corporation,Released
7,May 27 2022 10:08AM,238254,10,0085717745,ISDIN Corporation,Released
8,May 27 2022 10:08AM,238254,10,0085717758,ISDIN Corporation,Released
9,May 27 2022 10:08AM,238254,10,0085717759,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,238249,Released,1
31,238250,Released,20
32,238251,Released,29
33,238253,Released,2
34,238254,Released,35


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
238249,NaN,NaN,1.0
238250,NaN,NaN,20.0
238251,NaN,NaN,29.0
238253,NaN,NaN,2.0
238254,NaN,NaN,35.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
238159,0.0,0.0,1.0
238165,0.0,4.0,5.0
238168,0.0,0.0,1.0
238171,0.0,0.0,2.0
238177,0.0,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
238159,0.0,0,1
238165,0.0,4,5
238168,0.0,0,1
238171,0.0,0,2
238177,0.0,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,238159,0.0,0,1
1,238165,0.0,4,5
2,238168,0.0,0,1
3,238171,0.0,0,2
4,238177,0.0,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,238159,0.0,,1
1,238165,0.0,4,5
2,238168,0.0,,1
3,238171,0.0,,2
4,238177,0.0,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 10:08AM,238254,10,ISDIN Corporation
35,May 27 2022 10:03AM,238253,20,"Exact-Rx, Inc."
37,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.
66,May 27 2022 9:45AM,238248,10,ISDIN Corporation
104,May 27 2022 9:43AM,238250,10,"Methapharm, Inc."
124,May 27 2022 9:38AM,238249,10,ISDIN Corporation
125,May 27 2022 9:37AM,238247,10,ISDIN Corporation
156,May 27 2022 9:16AM,238243,10,ISDIN Corporation
183,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated"
184,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,May 27 2022 10:08AM,238254,10,ISDIN Corporation,0.0,,35
1,May 27 2022 10:03AM,238253,20,"Exact-Rx, Inc.",0.0,,2
2,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,0.0,,29
3,May 27 2022 9:45AM,238248,10,ISDIN Corporation,0.0,,38
4,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",0.0,,20
5,May 27 2022 9:38AM,238249,10,ISDIN Corporation,0.0,,1
6,May 27 2022 9:37AM,238247,10,ISDIN Corporation,0.0,,31
7,May 27 2022 9:16AM,238243,10,ISDIN Corporation,0.0,,27
8,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",0.0,,1
9,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",0.0,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 10:08AM,238254,10,ISDIN Corporation,35,
1,May 27 2022 10:03AM,238253,20,"Exact-Rx, Inc.",2,
2,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29,
3,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38,
4,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20,
5,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1,
6,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31,
7,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27,
8,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",1,
9,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 10:08AM,238254,10,ISDIN Corporation,35,
1,May 27 2022 10:03AM,238253,20,"Exact-Rx, Inc.",2,
2,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29,
3,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38,
4,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 10:08AM,238254,10,ISDIN Corporation,35.0,NaN
1,May 27 2022 10:03AM,238253,20,"Exact-Rx, Inc.",2.0,NaN
2,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29.0,NaN
3,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38.0,NaN
4,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 10:08AM,238254,10,ISDIN Corporation,35.0,0.0
1,May 27 2022 10:03AM,238253,20,"Exact-Rx, Inc.",2.0,0.0
2,May 27 2022 9:52AM,238251,20,Alumier Labs Inc.,29.0,0.0
3,May 27 2022 9:45AM,238248,10,ISDIN Corporation,38.0,0.0
4,May 27 2022 9:43AM,238250,10,"Methapharm, Inc.",20.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3096872,165.0,0.0
12,238210,1.0,0.0
15,714660,14.0,19.0
16,952825,9.0,2.0
18,238159,1.0,0.0
19,714623,3.0,18.0
20,1191038,44.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3096872,165.0,0.0
1,12,238210,1.0,0.0
2,15,714660,14.0,19.0
3,16,952825,9.0,2.0
4,18,238159,1.0,0.0
5,19,714623,3.0,18.0
6,20,1191038,44.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,165.0,0.0
1,12,1.0,0.0
2,15,14.0,19.0
3,16,9.0,2.0
4,18,1.0,0.0
5,19,3.0,18.0
6,20,44.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,165.0
1,12,Released,1.0
2,15,Released,14.0
3,16,Released,9.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Executing,0.0,0.0,19.0,2.0,0.0,18.0,5.0
Released,165.0,1.0,14.0,9.0,1.0,3.0,44.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Executing,0.0,0.0,19.0,2.0,0.0,18.0,5.0
1,Released,165.0,1.0,14.0,9.0,1.0,3.0,44.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Executing,0.0,0.0,19.0,2.0,0.0,18.0,5.0
1,Released,165.0,1.0,14.0,9.0,1.0,3.0,44.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()